In [4]:
import requests
import json

In [91]:
match_id = 2853522349

In [92]:
host = "https://api.opendota.com/api/matches/" + str(match_id)
data = {'match_id': match_id}

In [93]:
request = requests.get(host, data=data)

In [94]:
request.status_code

200

In [95]:
match_data = request.content
match_data = json.loads(match_data)

In [96]:
match_data.keys()

[u'replay_url',
 u'barracks_status_dire',
 u'match_id',
 u'barracks_status_radiant',
 u'cluster',
 u'replay_salt',
 u'first_blood_time',
 u'chat',
 u'dire_score',
 u'duration',
 u'radiant_win',
 u'skill',
 u'lobby_type',
 u'tower_status_dire',
 u'teamfights',
 u'series_type',
 u'objectives',
 u'version',
 u'cosmetics',
 u'leagueid',
 u'engine',
 u'radiant_score',
 u'human_players',
 u'start_time',
 u'match_seq_num',
 u'series_id',
 u'tower_status_radiant',
 u'negative_votes',
 u'picks_bans',
 u'positive_votes',
 u'radiant_xp_adv',
 u'region',
 u'game_mode',
 u'patch',
 u'radiant_gold_adv',
 u'players']

Fields of interest in match:
    1. version AND patch
    2. positive_votes and negative_votes
    3. first_blood_time
    4. barracks_status_radiant
    5. barracks_status_dire
    6. radiant_win
    7. possibly teamfights in the future

In [97]:
# Here we analyze a player
player1 = match_data['players'][0]

In [98]:
player1.keys()

[u'gold',
 u'actions',
 u'damage_inflictor_received',
 u'lh_t',
 u'damage_taken',
 u'kill_streaks',
 u'cosmetics',
 u'hero_id',
 u'kills_log',
 u'account_id',
 u'kills',
 u'kills_per_min',
 u'start_time',
 u'isRadiant',
 u'backpack_1',
 u'backpack_0',
 u'leaver_status',
 u'killed',
 u'stuns',
 u'gold_per_min',
 u'name',
 u'level',
 u'damage_inflictor',
 u'patch',
 u'item_4',
 u'item_5',
 u'item_2',
 u'item_3',
 u'item_0',
 u'lose',
 u'sen_left_log',
 u'obs_placed',
 u'match_id',
 u'kda',
 u'pings',
 u'gold_reasons',
 u'total_gold',
 u'item_uses',
 u'duration',
 u'sen',
 u'lobby_type',
 u'denies',
 u'killed_by',
 u'permanent_buffs',
 u'last_login',
 u'gold_t',
 u'xp_per_min',
 u'ability_uses',
 u'item_1',
 u'abandons',
 u'rune_pickups',
 u'assists',
 u'obs_left_log',
 u'xp_reasons',
 u'purchase_log',
 u'region',
 u'obs_log',
 u'times',
 u'obs',
 u'win',
 u'radiant_win',
 u'runes',
 u'game_mode',
 u'ability_upgrades_arr',
 u'party_id',
 u'hero_healing',
 u'life_state',
 u'xp_t',
 u'bench

https://github.com/kronusme/dota2-api/blob/master/data/heroes.json
https://github.com/kronusme/dota2-api/blob/master/data/items.json

Of interest from the player fields are:
    1. gold
    2. last_hints
    3. damage_taken
    4. isRadiant
    5. hero_id
    6. actions_per_min
    7. kills
    8. kills_per_min
    9. item_win for end state items
    10. kda
    11. assists
    12. deaths
    13. total_gold and total_xp
    14. denies
    15. item_0, item_1, item_2, ..., item_5
    16. lane_role
    17. lane_efficiency_pct
    18. solo_competitive_rank
    19. hero_damage
    20. hero_healing

In [50]:
def extractDotaInformation(matchJSON):
    matchInfo = {}
    matchInfo['version'] = matchJSON['version']
    matchInfo['patch'] = matchJSON['patch']
    matchInfo['positive_votes'] = matchJSON['positive_votes']
    matchInfo['negative_votes'] = matchJSON['negative_votes']
    matchInfo['first_blood_time'] = matchJSON['first_blood_time']
    matchInfo['barracks_status_radiant'] = matchJSON['barracks_status_radiant']
    matchInfo['barracks_status_dire'] = matchJSON['barracks_status_dire']
    matchInfo['tower_status_radiant'] = matchJSON['tower_status_radiant']
    matchInfo['tower_status_dire'] = matchJSON['tower_status_dire']
    matchInfo['radiant_win'] = matchJSON['radiant_win']
    matchInfo['duration'] = matchJSON['duration']
    # player info now
    # organize the heroes into positions and radiant vs dire

    for player in matchJSON['players']:
        if player['isRadiant']:
            matchInfo['hero_radiant' + str(player['lane_role'])] = player['hero_id']
            print player['lane_role']
            
            # rad.remove(player['lane_role'])
        else:
            matchInfo['hero_dire' + str(player['lane_role'])] = player['hero_id']
            print player['lane_role']
            
            # dire.remove(player['lane_role'])
    if len(rad) != 0 and len(rad) != len(dire):
        raise Exception()
    
    

In [51]:
entriesPerFile = 2
f = open('data/test_matches_'+str(2853522349)+'.txt', 'wb')

for x in range(match_id, match_id + 10):
    host = "https://api.opendota.com/api/matches/" + str(x)
    data = {'match_id': x}
    response = requests.get(host, data=data)
    if response.status_code != 200:
        raise Exception("Status code is {0}".format(response.status_code))
    match = json.loads(response.content)
    relevantMatch = extractDotaInformation(match)
    f.write(json.dumps(relevantMatch) + '\n')

f.close()

[u'purchase_time', u'lane_efficiency', u'gold', u'sentry_kills', u'lane_efficiency_pct', u'actions', u'damage_inflictor_received', u'lh_t', u'damage_taken', u'kill_streaks', u'cosmetics', u'hero_id', u'necronomicon_kills', u'kills_log', u'sentry_uses', u'account_id', u'kills', u'kills_per_min', u'start_time', u'isRadiant', u'backpack_1', u'backpack_0', u'leaver_status', u'killed', u'stuns', u'gold_per_min', u'name', u'level', u'actions_per_min', u'damage_inflictor', u'life_state_dead', u'patch', u'item_4', u'item_5', u'item_2', u'item_3', u'item_0', u'lose', u'sen_left_log', u'buyback_count', u'obs_placed', u'match_id', u'kda', u'pings', u'gold_reasons', u'total_gold', u'item_uses', u'duration', u'sen', u'lobby_type', u'denies', u'ancient_kills', u'killed_by', u'neutral_kills', u'permanent_buffs', u'last_login', u'gold_t', u'xp_per_min', u'ability_uses', u'item_1', u'abandons', u'rune_pickups', u'assists', u'obs_left_log', u'xp_reasons', u'purchase_log', u'purchase_ward_sentry', u'regi

KeyError: 'lane_role'